In [1]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN,PHONE_NUMBER,API_KEY_WAPI,K_NUMBER,J_NUMBER
import time
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime
from utils import request_wapi,get_forecast,create_df,send_message,get_date

In [2]:
query = 'La estrella'
api_key = API_KEY_WAPI

url = 'http://api.weatherapi.com/v1/forecast.json?key={}&q={}&days=1&aqi=no&alerts=no'.format(api_key,query)

In [3]:
response = requests.get(url).json()

In [4]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [5]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [6]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [7]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

'2023-06-21'

In [8]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [9]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Partly cloudy'

In [10]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

13.2

In [11]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [12]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

0

## Dataframe

In [13]:
def get_forecast(response,i):

    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0]
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temp = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    rain_prov = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,temp,rain,rain_prov

In [14]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = 'green'):

    datos.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<00:00, 24007.46it/s]


In [15]:
datos

[('2023-06-21', '00', 'Partly cloudy', 13.2, 0, 0),
 ('2023-06-21', '01', 'Partly cloudy', 13.0, 0, 0),
 ('2023-06-21', '02', 'Partly cloudy', 12.7, 0, 0),
 ('2023-06-21', '03', 'Partly cloudy', 12.5, 0, 0),
 ('2023-06-21', '04', 'Partly cloudy', 12.3, 0, 0),
 ('2023-06-21', '05', 'Clear', 12.1, 0, 0),
 ('2023-06-21', '06', 'Sunny', 12.0, 0, 0),
 ('2023-06-21', '07', 'Sunny', 16.9, 0, 0),
 ('2023-06-21', '08', 'Sunny', 20.5, 0, 0),
 ('2023-06-21', '09', 'Sunny', 23.1, 0, 0),
 ('2023-06-21', '10', 'Sunny', 25.1, 0, 0),
 ('2023-06-21', '11', 'Sunny', 29.4, 0, 0),
 ('2023-06-21', '12', 'Sunny', 30.4, 0, 0),
 ('2023-06-21', '13', 'Sunny', 30.6, 0, 0),
 ('2023-06-21', '14', 'Sunny', 30.8, 0, 0),
 ('2023-06-21', '15', 'Sunny', 30.5, 0, 0),
 ('2023-06-21', '16', 'Sunny', 29.7, 0, 0),
 ('2023-06-21', '17', 'Sunny', 26.6, 0, 0),
 ('2023-06-21', '18', 'Sunny', 20.6, 0, 0),
 ('2023-06-21', '19', 'Clear', 17.9, 0, 0),
 ('2023-06-21', '20', 'Patchy rain possible', 14.9, 1, 89),
 ('2023-06-21', '21'

In [16]:
col = ['Fecha','Hora','Condicion','Temp','Rain','Rain_prov']
df = pd.DataFrame(datos,columns=col)

In [17]:
df

,Fecha,Hora,Condicion,Temp,Rain,Rain_prov
0,2023-06-21,00,Partly cloudy,13.2,0,0
1,2023-06-21,01,Partly cloudy,13.0,0,0
2,2023-06-21,02,Partly cloudy,12.7,0,0
3,2023-06-21,03,Partly cloudy,12.5,0,0
4,2023-06-21,04,Partly cloudy,12.3,0,0
5,2023-06-21,05,Clear,12.1,0,0
6,2023-06-21,06,Sunny,12.0,0,0
7,2023-06-21,07,Sunny,16.9,0,0
8,2023-06-21,08,Sunny,20.5,0,0
9,2023-06-21,09,Sunny,23.1,0,0


In [18]:
df_rain = df[(df['Rain'] == 1)]
df_redu = df_rain[['Hora','Temp','Condicion']]
df_redu.set_index('Hora',inplace = True)

In [19]:
from datetime import datetime

datetime.now().time().hour

9

In [41]:
df_temp = df[['Hora','Temp','Condicion']]


In [42]:
current_hour = str(datetime.now().time().hour)
if len(current_hour) == 1:
    current_hour = current_hour.zfill(2)

df_temp = df_temp[df_temp['Hora'] == current_hour]

In [43]:
datetime.now().time().hour

9

In [44]:
df_temp

,Hora,Temp,Condicion
9,09,23.1,Sunny


In [22]:
df_temp.set_index('Hora',inplace = True)

In [24]:
df_redu

,Temp,Condicion
Hora,,
20,14.9,Patchy rain possible
21,14.3,Patchy rain possible


In [25]:

if df_redu.empty:
    print('no hay plan arrunchis, no va a llover :C')
else:
    print('wuju')

wuju


In [26]:
df_redu

,Temp,Condicion
Hora,,
20,14.9,Patchy rain possible
21,14.3,Patchy rain possible


In [27]:
k = K_NUMBER
j = J_NUMBER

## Mensaje SMS desde twilio

In [28]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client


In [29]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

In [30]:
'Hoy va a llover, abrígate bien :) \n\n\n ' + str(df_redu)


'Hoy va a llover, abrígate bien :) \n\n\n       Temp             Condicion\nHora                            \n20    14.9  Patchy rain possible\n21    14.3  Patchy rain possible'

In [33]:


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure

client = Client(account_sid, auth_token)
if df_redu.empty:
    template = '\n Hoy no hay arrunchis, no va a llover :c \n\n\n ' + str(df_redu) 
else:
    template = '\n Hoy va a llover en '+ query +', abrígate bien :) \n\n\n ' + str(df_temp)

message = client.messages \
    .create(
        
        body=template,
        from_=PHONE_NUMBER,
        to=k
     )

print('Mensaje enviado: ',message.sid, template)



Mensaje enviado:  SM35e06df29591babf5f2cbe569cee325f 
 Hoy va a llover en La estrella, abrígate bien :) 


       Temp             Condicion
Hora                            
20    14.9  Patchy rain possible
21    14.3  Patchy rain possible


In [34]:
'Hoy va a llover en '+ query +', abrígate bien :) \n\n\n ' + str(df_temp)

'Hoy va a llover en La estrella, abrígate bien :) \n\n\n Empty DataFrame\nColumns: [Temp, Condicion]\nIndex: []'

In [35]:
df_temp

,Temp,Condicion
Hora,,


In [160]:
df

,Fecha,Hora,Condicion,Temp,Rain,Rain_prov
0,2023-06-20,00,Clear,12.4,0,0
1,2023-06-20,01,Clear,12.1,0,0
2,2023-06-20,02,Partly cloudy,12.0,0,0
3,2023-06-20,03,Partly cloudy,11.8,0,0
4,2023-06-20,04,Partly cloudy,11.7,0,0
5,2023-06-20,05,Partly cloudy,11.6,0,0
6,2023-06-20,06,Partly cloudy,11.5,0,0
7,2023-06-20,07,Sunny,14.9,0,0
8,2023-06-20,08,Sunny,18.8,0,0
9,2023-06-20,09,Sunny,21.2,0,0
